In [1]:
%run /Users/hhg/Research/kinematic_lensing/code/BinnedFit/GaussFit_spec2D.py
%run /Users/hhg/Research/kinematic_lensing/code/BinnedFit/ChainTool.py
%run /Users/hhg/Research/kinematic_lensing/code/BinnedFit/RotationCurveFit.py
%run /Users/hhg/Research/kinematic_lensing/code/BinnedFit/ChainTool.py
%run /Users/hhg/Research/kinematic_lensing/code/BinnedFit/gamma.py


%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np

sys.path.append("/Users/hhg/Research/kinematic_lensing/code/BinnedFit/")
from binnedFit_utilities import *


sys.path.append("/Users/hhg/Research/kinematic_lensing/repo/kl_measurement/scripts/")
from parameter_class import fit_parameters
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import emcee
import chisq
from generate_TFcube import get_TFcube_pars, get_data_TFcube

from chainconsumer import ChainConsumer
%config InlineBackend.figure_format = 'retina'

dir_tfCube: /Users/hhg/Research/kinematic_lensing/repo/KLens


In [3]:
def gen_tfCube_data(sini, g1, g2):
    
    eint_thy = cal_e_int(sini=sini)
    eobs_thy = cal_e_obs(e_int=eint_thy, gamma_p=g1)
    theta_obs = cal_theta_obs(e_int=eint_thy, gamma_x=g2)

    slitAng_major_p = theta_obs
    slitAng_minor_p = theta_obs + np.pi/2.

    bestfit_e_obs = eobs_thy
    
    print(f"sini:{sini}, g1:{g1}, g2:{g2}")
    print("eobs_thy:", eobs_thy)
    print("slitAng_major_p:", slitAng_major_p*180./np.pi)
    print("slitAng_minor_p:", slitAng_minor_p*180./np.pi)
    
    update_TF, _ = get_TFcube_pars()
    update_TF['slitAngles'] = np.array([slitAng_major_p, slitAng_minor_p])
    
    update_TF['g1'] = g1
    update_TF['g2'] = g2
    update_TF['sini'] = sini
    update_TF['vcirc'] = 200. 

    update_TF['n_knots'] = 25.
    update_TF['knot_fraction'] = 0.0
    update_TF['throughput'] = 0.29
    update_TF['slitWidth'] = 0.05
    update_TF['psfFWHM'] = 0.5
    update_TF['pixScale'] = 0.1185 # 0.1185  # 0.032 

    update_TF['redshift'] = 0.2
    update_TF['Resolution'] = 6000.
    update_TF['nm_per_pixel'] = 0.033 #0.033 # 0.025
    update_TF['expTime'] = 3600.0 
    update_TF['linelist']['flux'][update_TF['linelist']['species'] == 'Halpha'] = 6.0e-24
    update_TF['lambda_min'] = (1 + update_TF['redshift']) * update_TF['linelist']['lambda'][update_TF['linelist']['species'] == 'Halpha'] - 2
    update_TF['lambda_max'] = (1 + update_TF['redshift']) * update_TF['linelist']['lambda'][update_TF['linelist']['species'] == 'Halpha'] + 2
    update_TF['norm'] = 0.
    update_TF['ngrid'] = 256
    update_TF['image_size'] = 128

    TF_pars, meta_parameters = get_TFcube_pars(update_TF)
    
    data, noise, obslambda = get_data_TFcube(TF_pars, meta_parameters, return_lambda=True)

    data_info = {}
    data_info['ModelCube'] = None
    data_info['ObsCube'] = None
    data_info['image'] = data['image']
    data_info['image_variance'] = noise['image']

    if len(TF_pars['slitAngles']) == 1:
        data_info['data'] = data['spectra'][0]
    else:
        #data_info['data'] = data['spectra'][0] # this line is to make it compatible with older branches... 
        data_info['data_major'] = data['spectra'][0]
        data_info['data_minor'] = data['spectra'][1]
    
    data_info['grid_lambda'] = obslambda

    extent =  TF_pars['image_size'] * TF_pars['pixScale']
    subGridPixScale = extent*1./TF_pars['ngrid']

    data_info['grid_pos']    = np.arange(-extent/2., extent/2., subGridPixScale)
    data_info['grid_Image']  = np.arange(-extent/2., extent/2., TF_pars['pixScale'])
    data_info['par_fid']     = TF_pars
    data_info['lambda_emit'] = 656.461 # Halpha [nm]
    
    data_info['eint_thy'] = eint_thy
    data_info['eobs_thy'] = eobs_thy
    data_info['theta_obs'] = theta_obs
    
    fname_out = f"tfCube_sini_{sini:.2f}_g1_{g1:.3f}_g2_{g2:.3f}.pkl"
    
    save_pickle(dir_tfCube_data+fname_out, data_info)

In [6]:
dir_tfCube_data = "/Users/hhg/Research/kinematic_lensing/data/mock_tfCube/"

g2_list = [0., 0.025, 0.05, 0.075, 0.1]
g1_list = [0., 0.025, 0.05, 0.075, 0.1]

sini=0.1

for g1 in g1_list:
    for g2 in g2_list:
        gen_tfCube_data(sini, g1, g2)

sini:0.1, g1:0.0, g2:0.0
eobs_thy: 0.00482315112540193
slitAng_major_p: 0.0
slitAng_minor_p: 90.0
returning:
lambda, observation, model, sky (the last three are (npix, npix, nspax) datacubes)
sini:0.1, g1:0.0, g2:0.025
eobs_thy: 0.00482315112540193
slitAng_major_p: 296.9831238094767
slitAng_minor_p: 386.9831238094767
returning:
lambda, observation, model, sky (the last three are (npix, npix, nspax) datacubes)
sini:0.1, g1:0.0, g2:0.05
eobs_thy: 0.00482315112540193
slitAng_major_p: 593.9662476189534
slitAng_minor_p: 683.9662476189535
returning:
lambda, observation, model, sky (the last three are (npix, npix, nspax) datacubes)
sini:0.1, g1:0.0, g2:0.075
eobs_thy: 0.00482315112540193
slitAng_major_p: 890.9493714284301
slitAng_minor_p: 980.94937142843
returning:
lambda, observation, model, sky (the last three are (npix, npix, nspax) datacubes)
sini:0.1, g1:0.0, g2:0.1
eobs_thy: 0.00482315112540193
slitAng_major_p: 1187.9324952379068
slitAng_minor_p: 1277.932495237907
returning:
lambda, obs